Instalari si importuri necesare

In [ ]:
!pip install pyphen

     |████████████████████████████████| 2.0 MB 5.2 MB/s 


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
import pyphen
import numpy as np
import pandas as pd
from nltk.corpus import wordnet
from dale_chall import DALE_CHALL
from nltk.tokenize import word_tokenize

Citirea datelor in dataframe

In [ ]:
dtypes = {"sentence": "string", "token": "string", "complexity": "float64"}
train = pd.read_excel('train.xlsx', dtype=dtypes, keep_default_na=False)
test = pd.read_excel('test.xlsx', dtype=dtypes, keep_default_na=False)

In [ ]:
print('train data: ', train.shape)
print('test data: ', test.shape)

Generarea de caracteristici legate de structura cuvantului pentru cuvantul tinta 

In [ ]:
def get_word_structure_features(word):
    features = []
    features.append(nr_syllables(word))
    features.append(is_dale_chall(word))
    features.append(length(word))
    features.append(nr_vowels(word))
    features.append(is_title(word))
    return np.array(features)

Generarea de caracteristici folosind Wordnet pentru cuvantul tinta

In [ ]:
def get_wordnet_features(word):
  features = []
  features.append(synsets(word))
  return np.array(features)

Apelul functiilor de generare de caracteristici

In [ ]:
def featurize(row):
    word = row['token']
    all_features = []
    all_features.extend(corpus_feature(row['corpus']))
    all_features.extend(get_word_structure_features(word))
    all_features.extend(get_wordnet_features(word))
    return np.array(all_features)

In [ ]:
def featurize_df(df):
    nr_of_features = len(featurize(df.iloc[0]))
    nr_of_examples = len(df)
    features = np.zeros((nr_of_examples, nr_of_features))
    for index, row in df.iterrows():
        row_ftrs = featurize(row)
        features[index, :] = row_ftrs
    return features

Generarea de caracteristici pentru setul de train

In [ ]:
X_train = featurize_df(train)
y_train = train['complex'].values

Generarea de caracteristici pentru setul de test

In [ ]:
X_test = featurize_df(test)

In [ ]:
for nb in range(1, 8, 2):
    model = KNeighborsClassifier(n_neighbors=nb)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)